In [1]:
from keras.preprocessing.image import img_to_array, load_img, array_to_img
import os
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import StandardScaler


Using TensorFlow backend.


In [4]:
train_feature_data = pd.read_csv("train.csv")
test_feature_data = pd.read_csv("test.csv")

ID = train_feature_data.pop("id")

train_labels = train_feature_data.pop('species')
le = preprocessing.LabelEncoder()
train_labels = le.fit(train_labels).transform(train_labels) 

# standardize the data by setting the mean to 0 and std to 1
train_feature_data = StandardScaler().fit(train_feature_data).transform(train_feature_data)

X_train, X_val, y_train, y_val = train_test_split(train_feature_data, train_labels, test_size=.3, random_state=40, stratify = train_labels)
y_train = (np.arange(99) == y_train[:,None]).astype(np.float32)
y_val = (np.arange(99) == y_val[:,None]).astype(np.float32)

X_train = X_train.astype(np.float32)
X_val = X_val.astype(np.float32)

print("Training Image Data", X_train.shape, y_train.shape)
print("Validated Image Data", X_val.shape, y_val.shape)

Training Image Data (693, 192) (693, 99)
Validated Image Data (297, 192) (297, 99)


In [5]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.01)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


split_by_half = lambda x,k : int(x/2**k)

In [6]:
batch_size = 32
hidden_nodes = 2048

graph = tf.Graph()
with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, 192))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, 99))
    tf_valid_dataset = tf.constant(X_val)
    tf_valid_labels = tf.constant(y_val)

    # tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = weight_variable([192, hidden_nodes])
    layer1_biases = bias_variable([hidden_nodes])
    layer4_weights = weight_variable([hidden_nodes, 99])
    layer4_biases = bias_variable([99])
    
    keep_prob = tf.placeholder("float")
    
    # Model with dropout
    def model(data, proba=keep_prob):
        layer1 = tf.matmul(data, layer1_weights) + layer1_biases
        hidden1 = tf.nn.dropout(tf.nn.relu(layer1), proba)  # dropout on hidden layer
        return tf.matmul(hidden1, layer4_weights) + layer4_biases
    
    # Training computation.
    logits = model(tf_train_dataset, keep_prob)
    loss = tf.reduce_sum(tf.multiply(tf.log(tf.clip_by_value(tf.nn.softmax(logits), 1e-10, 1.0)), tf_train_labels))/-32.0

    # Optimizer.
    optimizer = tf.train.RMSPropOptimizer(1e-4).minimize(loss)


    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset,1.0))
    loss_v = tf.reduce_sum(tf.multiply(tf.log(tf.clip_by_value(valid_prediction, 1e-10, 1.0)), tf_valid_labels))/-(99*3)
  
    # test_prediction = tf.nn.softmax(model(tf_test_dataset,1.0))


In [7]:
# Re-define the function to include the keep probability
res_final = {}
def run_session(num_epochs, name, k_prob=1.0):

    with tf.Session(graph=graph) as session:
        merged = tf.merge_all_summaries()  
        writer = tf.train.SummaryWriter("/tmp/tensorflowlogs", session.graph)
        tf.initialize_all_variables().run()
        print("Initialized")
        for epoch in range(num_epochs):
            offset = (epoch * batch_size) % (y_train.shape[0] - batch_size)
            batch_data = X_train[offset:(offset + batch_size), :]
            batch_labels = y_train[offset:(offset + batch_size), :]
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : k_prob}
            _, l, predictions, l_v = session.run([optimizer, loss, train_prediction, loss_v], feed_dict=feed_dict)
#            if (epoch % 500 == 0):
#                print("Minibatch loss at epoch {}: {}".format(epoch, l))
#                print("Minibatch loss at epoch {}: {}".format(epoch, l_v))
#                print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
#                print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), y_val)))
        
        # print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))
        # test_preds[name] = test_prediction.eval().ravel()
        res_final["val_preds"] = valid_prediction.eval()
        print("rigit", np.argmax(y_val, 1)[np.argmax(res_final["val_preds"], 1) != np.argmax(y_val, 1)])
        print("pred", np.argmax(res_final["val_preds"], 1)[np.argmax(res_final["val_preds"], 1) != np.argmax(y_val, 1)])
        

In [9]:
for i in range(40):
    X_train, X_val, y_train, y_val = train_test_split(train_feature_data, train_labels, test_size=.1, random_state=i, stratify = train_labels)
    y_train = (np.arange(99) == y_train[:,None]).astype(np.float32)
    y_val = (np.arange(99) == y_val[:,None]).astype(np.float32)

    X_train = X_train.astype(np.float32)
    X_val = X_val.astype(np.float32)
    
    batch_size = 32
    hidden_nodes = 2048

    graph = tf.Graph()
    with graph.as_default():

        # Input data.
        tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, 192))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, 99))
        tf_valid_dataset = tf.constant(X_val)
        tf_valid_labels = tf.constant(y_val)

        # tf_test_dataset = tf.constant(test_dataset)

        # Variables.
        layer1_weights = weight_variable([192, hidden_nodes])
        layer1_biases = bias_variable([hidden_nodes])
        layer4_weights = weight_variable([hidden_nodes, 99])
        layer4_biases = bias_variable([99])
    
        keep_prob = tf.placeholder("float")
    
        # Model with dropout
        def model(data, proba=keep_prob):
            layer1 = tf.matmul(data, layer1_weights) + layer1_biases
            hidden1 = tf.nn.dropout(tf.nn.relu(layer1), proba)  # dropout on hidden layer
            return tf.matmul(hidden1, layer4_weights) + layer4_biases
    
       # Training computation.
        logits = model(tf_train_dataset, keep_prob)
        loss = tf.reduce_sum(tf.multiply(tf.log(tf.clip_by_value(tf.nn.softmax(logits), 1e-10, 1.0)), tf_train_labels))/-32.0

        # Optimizer.
        optimizer = tf.train.RMSPropOptimizer(1e-4).minimize(loss)


        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = tf.nn.softmax(model(tf_valid_dataset,1.0))
        loss_v = tf.reduce_sum(tf.multiply(tf.log(tf.clip_by_value(valid_prediction, 1e-10, 1.0)), tf_valid_labels))/-(99*3)
  
        # test_prediction = tf.nn.softmax(model(tf_test_dataset,1.0))
    
    run_session(5000, "Deep_NN", 0.5)
    

Instructions for updating:
Please switch to tf.summary.merge_all.
Instructions for updating:
Please switch to tf.summary.FileWriter. The interface and behavior is the same; this is just a rename.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
rigit [80]
pred [71]
Instructions for updating:
Please switch to tf.summary.merge_all.
Instructions for updating:
Please switch to tf.summary.FileWriter. The interface and behavior is the same; this is just a rename.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
rigit [38]
pred [37]
Instructions for updating:
Please switch to tf.summary.merge_all.
Instructions for updating:
Please switch to tf.summary.FileWriter. The interface and behavior is the same; this is just a rename.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
rigit []
pred []
Instructions for updating:
Please switch to tf.summary.merge_all.
Instructions for updating:
Pl

In [ ]:
71-80 9
37-38 5
28-69 
71-80
54-75 69-55 66-64
54-75 28-29
92-73
80-71
31-68
28-29 80-71
51-74 69-55
69-55 37-38
71-80
69-55
28-29 71-80
92-73
54-75 66-44
46-75 77-97 28-29
71-80 69-55
54-75
37-38
37-38 77-97
92-73
54-75 6
66-44
37-38 54-75 71-80 77-97
66-44 28-29 5 80-71

27

In [11]:
le.inverse_transform([71, 80])

array(['Quercus_Palustris', 'Quercus_Shumardii'], dtype=object)